In [ ]:
""" Задачи """

In [37]:
import pandas as pd
from datetime import datetime
from scipy import stats
import os

In [3]:
def read_file(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'Файл {file_name} не найден')
    else:
        return pd.read_csv(file_name)

Задача 1. Необходимо сравнить 2 датасета - один до изменений на сайте, другой после. И проверить значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23) В качестве ответа вывести p-value

In [10]:
df_sales = read_file("df_sales.csv")
df_sales['date'] = pd.to_datetime(df_sales['date'])
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [9]:
df_users = read_file('experiment_users.csv')
df_users.head()

,user_id,pilot
0,0ffc65,0
1,b962b9,0
2,7ea63f,0
3,7f9a61,0
4,459e55,0


In [17]:
date_start = datetime(2022, 3, 16)
date_end = datetime(2022, 3, 23)

df_metrics = df_sales[(df_sales['date']>=date_start) & (df_sales['date'] < date_end)].groupby('user_id')['price'].agg('sum').reset_index()

In [27]:
df = pd.merge(df_users, df_metrics, on='user_id', how='left')

In [34]:
df['price'] = df['price'].fillna(0)
df.head()

,user_id,pilot,price
0,0ffc65,0,0.0
1,b962b9,0,0.0
2,7ea63f,0,0.0
3,7f9a61,0,0.0
4,459e55,0,2160.0


In [45]:
data_control = df[df['pilot']==0]['price']
data_pilot = df[df['pilot']==1]['price']

p_value = stats.ttest_ind(data_control, data_pilot).pvalue
print(f'p_value = {p_value:0.3f}')

p_value = 0.199


 Задача 2. Среднее время между покупками.
 - Нужно оценить среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.
В качестве ответа введите среднее количество дней между покупками, округлённое до целого значения.

In [65]:
df_sales = read_file('df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [57]:
#выделяем отдельный датафрейм с клиентами, у которых было больше 2-ух покупок
#active_users = df_sales.groupby('user_id').filter(lambda x: len(x)>2)

In [77]:
df = df[df['count'] >= 2]
#Расчёт общего времени между первой и последней покупкой для каждого пользователя
df['delta'] = (df['max'] - df['min']).dt.total_seconds()
#Суммарное время между всеми покупками всех пользователей
sum_delta = df['delta'].sum()
#Расчёт общего количества периодов между покупками
count_periods = df['count'].sum() - len(df)
#Вычисление среднего интервала в днях
answer = sum_delta / count_periods / 3600 / 24
print('answer:', int(round(answer)))

answer: 17
